In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

import datetime

from wordcloud import WordCloud
from wordcloud import STOPWORDS

import math
import numpy as np

In [220]:
data = sqlite3.connect('/Users/cp/Documents/dsi/8a_kaggle/database.sqlite')
ascents_df = pd.read_sql_query("SELECT * FROM ascent where name = 'Kerveros' and climb_type = 0 ", data)
ascents_df['date'] = pd.to_datetime(ascents_df['date'], unit = 's')
# ascents_df['date'] = ascents_df.date.dt.round('1d')

In [221]:
# ascents_df['name'] = ascents_df['name'].str.lower()
ascents_df['rec_date'] = pd.to_datetime(ascents_df['date'], unit = 's')

In [222]:
ascents_df.comment.value_counts()

                                                                                                                                                       644
Amazing climb. Bit of everything.                                                                                                                        4
Repeat.                                                                                                                                                  3
6c+/7a                                                                                                                                                   3
one of the best!                                                                                                                                         2
                                                                                                                                                      ... 
tricky one, tufas pockets, the whole 9 yards. great                   

In [223]:
ascents_df.crag.value_counts()

Kalymnos    914
Samos         1
Name: crag, dtype: int64

In [224]:
ascents_df.columns

Index(['id', 'user_id', 'grade_id', 'notes', 'raw_notes', 'method_id',
       'climb_type', 'total_score', 'date', 'year', 'last_year', 'rec_date',
       'project_ascent_date', 'name', 'crag_id', 'crag', 'sector_id', 'sector',
       'country', 'comment', 'rating', 'description', 'yellow_id', 'climb_try',
       'repeat', 'exclude_from_ranking', 'user_recommended', 'chipped'],
      dtype='object')

In [225]:
ascents_df.shape

(915, 28)

In [171]:
# ascents_df.to_csv('/Users/cp/Documents/dsi/8a_kaggle/boulders_full.csv', index= False)
col_list = ['userAvatar', 'userName', 'userSlug', 'date', 'difficulty', 'isHard', 
       'isSoft', 'type', 'comment', 'traditional', 'project', 'rating', 
       'userPrivate', 'zlagGradeIndex', 'zlaggableName', 'zlaggableSlug', 
       'cragSlug', 'cragName', 'countrySlug', 'countryName', 'areaSlug', 
       'areaName', 'sectorSlug', 'sectorName', 'category', 'recommended', 
       'firstAscent', 'secondGo', 'isBoltedByMe', 'isOverhang', 'isVertical', 
       'isSlab', 'isRoof', 'isAthletic', 'isEndurance', 'isCrimpy', 'isCruxy', 
       'isSloper', 'isTechnical', 'isDanger', 'chipped', 'withKneepad', 
       'badAnchor', 'badBolts', 'highFirstBolt', 'looseRock', 
       'badClippingPosition'] 

In [226]:
# ascents_df.head()
scraped_df = pd.read_csv('/Volumes/Backup Plus/boulder_csv/routes/Kerveros.csv', names = col_list)

In [227]:
scraped_df.columns


Index(['userAvatar', 'userName', 'userSlug', 'date', 'difficulty', 'isHard',
       'isSoft', 'type', 'comment', 'traditional', 'project', 'rating',
       'userPrivate', 'zlagGradeIndex', 'zlaggableName', 'zlaggableSlug',
       'cragSlug', 'cragName', 'countrySlug', 'countryName', 'areaSlug',
       'areaName', 'sectorSlug', 'sectorName', 'category', 'recommended',
       'firstAscent', 'secondGo', 'isBoltedByMe', 'isOverhang', 'isVertical',
       'isSlab', 'isRoof', 'isAthletic', 'isEndurance', 'isCrimpy', 'isCruxy',
       'isSloper', 'isTechnical', 'isDanger', 'chipped', 'withKneepad',
       'badAnchor', 'badBolts', 'highFirstBolt', 'looseRock',
       'badClippingPosition'],
      dtype='object')

In [228]:
scraped_df.comment.value_counts()

Repeat.                                     6
Amazing climb. Bit of everything.           4
one of the best!                            2
6c+/7a                                      2
OS                                          1
                                           ..
Putting up draws. Nice route.               1
feels quite hard for 7a                     1
Already tried 2 years ago, warmup today.    1
A whole lotta tufas right off the plane.    1
new- more  6c+?                             1
Name: comment, Length: 297, dtype: int64

In [229]:
scraped_df.areaName.value_counts()

Kalymnos    1159
Name: areaName, dtype: int64

In [230]:
ascents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    915 non-null    int64         
 1   user_id               915 non-null    int64         
 2   grade_id              915 non-null    int64         
 3   notes                 915 non-null    object        
 4   raw_notes             915 non-null    int64         
 5   method_id             915 non-null    int64         
 6   climb_type            915 non-null    int64         
 7   total_score           915 non-null    int64         
 8   date                  915 non-null    datetime64[ns]
 9   year                  915 non-null    int64         
 10  last_year             915 non-null    int64         
 11  rec_date              915 non-null    datetime64[ns]
 12  project_ascent_date   915 non-null    int64         
 13  name                

In [231]:
# scraped_df.date.to_datetime
# scraped_df['date'] = pd.to_datetime(scraped_df['date'], unit = 's', errors = 'coerce')

In [232]:
scraped_df.head()

,userAvatar,userName,userSlug,date,difficulty,isHard,isSoft,type,comment,traditional,...,isSloper,isTechnical,isDanger,chipped,withKneepad,badAnchor,badBolts,highFirstBolt,looseRock,badClippingPosition
0,p/gallery/tp3vldka.jpg,Yuliia Yurasova,yuliia-yurasova,2021-06-28T12:00:00+00:00,7a,False,False,rp,NaN,False,...,False,False,False,False,False,False,False,False,False,False
1,NaN,Kuba Barzycki,kuba-barzycki,2021-06-07T12:00:00+00:00,7a,False,False,os,had to fight a bit for OS,False,...,False,False,False,False,False,False,False,False,False,False
2,NaN,Pawel Pospiech,pawel-pospiech,2021-06-07T12:00:00+00:00,7a,False,False,f,NaN,False,...,False,False,False,False,False,False,False,False,False,False
3,gallery/38327.jpg,Erik van den Goor,erik-van-den-goor,2021-06-05T12:00:00+00:00,7a,False,False,rp,NaN,False,...,False,False,False,False,False,False,False,False,False,False
4,gallery/1312.jpg,Monica Zeilemaker (f),monica-zeilemaker-f,2021-06-05T12:00:00+00:00,7a,False,False,f,After 5 months lockdown. Not bad...:-),False,...,False,False,False,False,False,False,False,False,False,False


In [233]:
scraped_df['date'] = pd.to_datetime(scraped_df['date']).dt.tz_localize(None)

In [234]:
scraped_df.tail(20)

,userAvatar,userName,userSlug,date,difficulty,isHard,isSoft,type,comment,traditional,...,isSloper,isTechnical,isDanger,chipped,withKneepad,badAnchor,badBolts,highFirstBolt,looseRock,badClippingPosition
1139,gallery/2420.JPG,Jussi Isoaho,jussi-isoaho,2004-10-02,7a,False,False,os,Sustained 6c+/7a,False,...,False,False,False,False,False,False,False,False,False,False
1140,NaN,Nick Gazos,nick-gazos,2004-10-02,7a,False,False,os,NaN,False,...,False,False,False,False,False,False,False,False,False,False
1141,NaN,Richard Hardicre,richard-hardicre,2004-09-28,7a,False,False,os,NaN,False,...,False,False,False,False,False,False,False,False,False,False
1142,NaN,Staffan Karlsson,staffan-karlsson,2004-09-20,7a,False,False,f,NaN,False,...,False,False,False,False,False,False,False,False,False,False
1143,gallery/2782.jpg,Steve G,steve-g,2004-09-12,7a,False,False,os,NaN,False,...,False,False,False,False,False,False,False,False,False,False
1144,gallery/14570.JPG,Cinkshy Ena Numpa,cinkshy-ena-numpa,2004-09-10,7a,False,False,os,NaN,False,...,False,False,False,False,False,False,False,False,False,False
1145,gallery/3188.jpg,Tom Atle Bordevik,tom-atle-bordevik,2004-09-01,7a,False,False,os,Pumpy and awesome,False,...,False,False,False,False,False,False,False,False,False,False
1146,NaN,Thanasis Htenas,thanasis-htenas,2004-07-27,7b,False,False,os,NaN,False,...,False,False,False,False,False,False,False,False,False,False
1147,gallery/4121.jpg,Jan Broeders,jan-broeders,2004-06-12,7a,False,False,rp,slip,False,...,False,False,False,False,False,False,False,False,False,False
1148,gallery/848.JPeG,Erik Lucas,erik-lucas,2004-06-12,7a,False,False,f,NaN,False,...,False,False,False,False,False,False,False,False,False,False


In [235]:
ascents_df.date = ascents_df.date.dt.round('1d')

In [236]:
scraped_df.shape

(1159, 47)

In [237]:
ascents_df.head()

,id,user_id,grade_id,notes,raw_notes,method_id,climb_type,total_score,date,year,...,country,comment,rating,description,yellow_id,climb_try,repeat,exclude_from_ranking,user_recommended,chipped
0,71068,2572,31,,0,2,0,353,2002-08-01,2002,...,GRC,,0,,0,0,0,0,0,0
1,81081,1024,31,,0,2,0,353,2002-09-18,2002,...,GRC,,3,,0,0,0,0,0,0
2,129937,4178,31,,0,3,0,445,2002-06-01,2002,...,GRC,SAMOS-ISL,0,,0,0,0,0,0,0
3,144183,2682,31,,0,3,0,445,2003-08-06,2003,...,GRC,,2,,0,0,0,0,0,0
4,144203,2545,31,,0,2,0,353,2003-08-06,2003,...,GRC,,2,,0,0,0,0,0,0


In [239]:
ascents_df[ascents_df.user_id ==2682]

,id,user_id,grade_id,notes,raw_notes,method_id,climb_type,total_score,date,year,...,country,comment,rating,description,yellow_id,climb_try,repeat,exclude_from_ranking,user_recommended,chipped
3,144183,2682,31,,0,3,0,445,2003-08-06,2003,...,GRC,,2,,0,0,0,0,0,0


In [240]:
ascents_df.shape

(915, 28)

In [241]:
ascents_df.date.drop_duplicates(inplace = True)

In [242]:
scraped_df.date.drop_duplicates(inplace = True)

In [117]:
# scraped_df[scraped_df.userAvatar=='gallery/21931.jpg']

,userAvatar,userName,userSlug,date,difficulty,isHard,isSoft,type,comment,traditional,...,isSloper,isTechnical,isDanger,chipped,withKneepad,badAnchor,badBolts,highFirstBolt,looseRock,badClippingPosition
503,gallery/21931.jpg,Miha Jarm,miha-jarm,2013-09-20,7a,False,False,os,NaN,False,...,False,False,False,False,False,False,False,False,False,False


In [98]:
ascents_df.comment.value_counts()

                                                                                                                                                                            476
repeat...                                                                                                                                                                     2
Fun and friendly after the marathons in the Grande Grotta.                                                                                                                    1
3° giro                                                                                                                                                                       1
Muchas Gracias a David por la paciencia, por me asegurar y por las palabras de incentivo! Valeu Carol pela eterna parceria em kalymnos! Thanks Alon for the good energy!      1
                                                                                                                        

In [243]:
scraped_df.date = scraped_df.date.dt.round('1d')

In [244]:
ascents_df[ascents_df.comment =='']

,id,user_id,grade_id,notes,raw_notes,method_id,climb_type,total_score,date,year,...,country,comment,rating,description,yellow_id,climb_try,repeat,exclude_from_ranking,user_recommended,chipped
0,71068,2572,31,,0,2,0,353,2002-08-01,2002,...,GRC,,0,,0,0,0,0,0,0
1,81081,1024,31,,0,2,0,353,2002-09-18,2002,...,GRC,,3,,0,0,0,0,0,0
3,144183,2682,31,,0,3,0,445,2003-08-06,2003,...,GRC,,2,,0,0,0,0,0,0
4,144203,2545,31,,0,2,0,353,2003-08-06,2003,...,GRC,,2,,0,0,0,0,0,0
6,158128,1728,49,,0,2,0,753,2003-09-20,2003,...,GRC,,3,,255,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,4816725,17897,49,,0,3,0,845,2015-08-30,2015,...,GRC,,0,,255,0,0,0,0,0
905,4824219,56675,49,,0,3,0,845,2016-10-26,2016,...,GRC,,0,,255,0,0,0,0,0
907,4833476,3943,49,,0,2,0,753,2011-09-27,2011,...,GRC,,0,,255,0,0,0,0,0
912,4900898,47451,49,,0,3,0,845,2017-09-05,2017,...,GRC,,0,,255,0,0,0,0,0


In [245]:
new_df = pd.merge(ascents_df, scraped_df, how = 'inner', left_on = 'date', right_on = 'date')

In [246]:
new_df.head()

,id,user_id,grade_id,notes,raw_notes,method_id,climb_type,total_score,date,year,...,isSloper,isTechnical,isDanger,chipped_y,withKneepad,badAnchor,badBolts,highFirstBolt,looseRock,badClippingPosition
0,71068,2572,31,,0,2,0,353,2002-08-01,2002,...,False,False,False,False,False,False,False,False,False,False
1,81081,1024,31,,0,2,0,353,2002-09-18,2002,...,False,False,False,False,False,False,False,False,False,False
2,144183,2682,31,,0,3,0,445,2003-08-06,2003,...,False,False,False,False,False,False,False,False,False,False
3,144183,2682,31,,0,3,0,445,2003-08-06,2003,...,False,False,False,False,False,False,False,False,False,False
4,144203,2545,31,,0,2,0,353,2003-08-06,2003,...,False,False,False,False,False,False,False,False,False,False


In [247]:
new_df.shape

(1505, 74)

In [248]:
compare_df = new_df[['user_id', 'userAvatar']]

In [249]:
compare_df.head(100)

,user_id,userAvatar
0,2572,NaN
1,1024,gallery/1024.jpg
2,2682,NaN
3,2682,gallery/2682.jpg
4,2545,NaN
...,...,...
95,14570,gallery/14570.JPG
96,10268,gallery/10268.jpg
97,249,gallery/249.jpg
98,14630,p/gallery/g55jn0q1.jpg


In [250]:
compare_df.nunique()

user_id       890
userAvatar    505
dtype: int64

In [251]:
scraped_df.zlaggableName.value_counts()

Kerveros    1159
Name: zlaggableName, dtype: int64

In [252]:
compare_df.shape

(1505, 2)

In [254]:
compare_df.tail(100)

,user_id,userAvatar
1405,26250,gallery/39431.jpg
1406,56992,NaN
1407,56992,NaN
1408,37858,gallery/37858.jpg
1409,16356,gallery/16356.gif
...,...,...
1500,45626,gallery/48478.jpg
1501,45626,gallery/45626.jpg
1502,48478,gallery/48478.jpg
1503,48478,gallery/45626.jpg
